In [3]:
# --------------------------------------------------------------------------------------
# 📌 Module 1: Importing Required Libraries
# --------------------------------------------------------------------------------------
import cv2  # OpenCV for video capture and image processing
import datetime  # For timestamping snapshots
import os  # For folder creation
import time  # To implement cooldown timer between snapshots

# --------------------------------------------------------------------------------------
# 📌 Module 2: Folder Setup for Captured Snapshots
# --------------------------------------------------------------------------------------
# Creating a folder named 'Captured' if it does not exist
if not os.path.exists("Captured"):
    os.makedirs("Captured")
    print("[INFO] 'Captured' folder created successfully.")

# --------------------------------------------------------------------------------------
# 📌 Module 3: Webcam Initialization
# --------------------------------------------------------------------------------------
# Open the default webcam (camera index 0)
video = cv2.VideoCapture(0)
print("[INFO] Webcam access granted. Starting video feed...")

# --------------------------------------------------------------------------------------
# 📌 Module 4: Initial Setup for Motion Detection
# --------------------------------------------------------------------------------------
previous_frame = None  # To store the last processed frame for comparison
last_capture_time = 0  # To manage cooldown between captures
cooldown_seconds = 5   # ⏱ Cooldown time (in seconds) between snapshot captures

# --------------------------------------------------------------------------------------
# 📌 Module 5: Real-Time Frame Capture and Motion Detection Loop
# --------------------------------------------------------------------------------------
while True:
    check, frame = video.read()  # Capture a new frame from webcam

    # Step 1: Convert to Grayscale and apply Gaussian Blur to reduce noise
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # Step 2: Handle the very first frame (used as a baseline)
    if previous_frame is None:
        previous_frame = gray
        continue

    # Step 3: Compute the absolute difference between current and previous frame
    delta_frame = cv2.absdiff(previous_frame, gray)
    previous_frame = gray  # Update previous frame to current for next iteration

    # Step 4: Thresholding to highlight differences and dilate for better contour detection
    thresh = cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Step 5: Find contours of the thresholded areas
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    motion_detected = False  # Flag to check if motion is found in this frame

    for contour in contours:
        # Filter out small changes that are likely noise (area threshold)
        if cv2.contourArea(contour) < 7500:
            continue
        motion_detected = True

        # Draw bounding box around detected motion
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # ----------------------------------------------------------------------------------
    # 📌 Module 6: Snapshot Capture with Cooldown
    # ----------------------------------------------------------------------------------
    current_time = time.time()
    if motion_detected and (current_time - last_capture_time > cooldown_seconds):
        # Step 1: Generate timestamp
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

        # Step 2: Overlay timestamp on video frame
        cv2.putText(frame, timestamp, (10, frame.shape[0] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        # Step 3: Save frame as image in 'Captured' folder
        filename = f"Captured/motion_{timestamp}.jpg"
        cv2.imwrite(filename, frame)

        # Step 4: Notify user in console
        print(f"[INFO] Motion detected! Image saved as {filename}")

        # Step 5: Update cooldown timer
        last_capture_time = current_time

    # ----------------------------------------------------------------------------------
    # 📌 Module 7: Display Live Video Feed with Motion Boxes
    # ----------------------------------------------------------------------------------
    cv2.imshow("Motion Detector", frame)

    # Exit the loop when 'q' is pressed
    key = cv2.waitKey(1)
    if key == ord('q'):
        print("[INFO] Quitting program as 'q' was pressed.")
        break

# --------------------------------------------------------------------------------------
# 📌 Module 8: Cleanup - Release Webcam and Close All Windows
# --------------------------------------------------------------------------------------
video.release()
cv2.destroyAllWindows()
print("[INFO] Camera released and all windows closed.")


[INFO] Webcam access granted. Starting video feed...
[INFO] Motion detected! Image saved as Captured/motion_2025-06-06_10-27-12.jpg
[INFO] Motion detected! Image saved as Captured/motion_2025-06-06_10-27-18.jpg
[INFO] Motion detected! Image saved as Captured/motion_2025-06-06_10-27-25.jpg
[INFO] Motion detected! Image saved as Captured/motion_2025-06-06_10-27-32.jpg
[INFO] Motion detected! Image saved as Captured/motion_2025-06-06_10-27-43.jpg
[INFO] Motion detected! Image saved as Captured/motion_2025-06-06_10-27-48.jpg
[INFO] Quitting program as 'q' was pressed.
[INFO] Camera released and all windows closed.
